In [144]:
from musical_festival_lineup.musical_festival_lineup import MusicalFestivalSolution
from musical_festival_lineup.musical_festival_data import MusicalFestivalData
from musical_festival_lineup.utils import visualize_musical_lineup
import numpy as np
import time

In [130]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### 1. Structuring the problem

The  objective  is  to  design  the  optimal  festival  lineup  by  scheduling  artists  across  stages  and 
time  slots while:
- Maximizing  prime  slot  popularity
- Ensuring  genre  diversity  among  stages 
- Minimizing  fan conflicts  at each time slot. 

__`Details`__

- **Artists**: An  artist is  characterized  by  a  popularity  score  (from  0  to  100) a genre  and  fan  base.
    <br>This information with be on __`artists`__ from musical_festival_data

- **Conflicts**:   with  other  artists  (score  from  0  to  1).  Conflicts  arise  when  artists  with  overlapping 
fan  bases  perform  at  the  same  time  but  on  different  stages,  which  can  negatively  impact 
attendance.
    <br>This information will be on __`conflits`__ from musical_festival_data


### 2. Defining a  solution/individual

Each  individual  represents  a  festival  lineup,  specifying  which  artist  plays  on  which  stage  and 
at  which  time  slot. 
These  are  the  constraints  : 
- Each artist is assigned to exactly one stage and slot 
- All time slots start and end at the same time. 
- All stages have the same amount of slots
- These artists should be distributed across  **5 stages  with  7 time slots each**

__`Important`__ 
<br>Any  lineup  where  an  artist  is  assigned  to  **multiple  time  slots  or 
left  unassigned** is  not  part  of  the  search  space  and  is  not  considered  a  solution.  It  is 
forbidden to generate such an arrangement during evolution. 


__`we propose a  having  a solution as`__:

- A string of 5 (stages)*7(slots)= 35 numbers of **1** to **35**
- As for the genetic algorithms, we should have string of the same size, we will be appending a 0 to 1 digit numbers.
- So the solution will be a string of 35 * 2 (two character)= so will have i string of 70

Here a individual of our problem:

In [ ]:
data=MusicalFestivalData()


In [89]:
# Python code to show a random initialized solution
lineup=MusicalFestivalSolution(data=data)
lineup.repr

'0719180922120231240032173408010630102923211516111404282003261325270533'

- Having that, we have to options:
- 1. The first 7 numbers represent the stage 1 , having each of the 7 representing the slots in  order (1 to 7) by the slot?
- 2. Or the first 5 would represent the slot 1 (first time slot) , having the each of the 5 representing the stages in order (1 to 5)?

For that we choose the option 2, because how our fitness will be defined. It is well explained in the session 3, when defining fitness


In [91]:
visualize_musical_lineup(lineup.repr,data.artists)

,Slot,Stage 1,Stage 2,Stage 3,Stage 4,Stage 5
0,Slot 1,7: Static Mirage|Rock|94,19: Astral Tide|Electronic|69,18: Velvet Underground|Rock|72,9: Deep Resonance|Jazz|90,22: Quantum Beat|Hip-Hop|96
1,Slot 2,12: Blue Horizon|Pop|51,2: Velvet Pulse|Jazz|35,31: The Jazz Nomads|Jazz|64,24: Shadow Cadence|Jazz|66,0: Midnight Echo|Rock|75
2,Slot 3,32: The Bassline Architects|Hip-Hop|61,17: Nightfall Sonata|Classical|84,34: Parallel Dimension|Electronic|58,8: Crimson Harmony|Classical|20,1: Solar Flare|Electronic|78
3,Slot 4,6: Aurora Skies|Pop|75,30: Turbo Vortex|Rock|53,10: The Wandering Notes|Jazz|84,29: Harmonic Dissonance|Classical|96,23: Electric Serpents|Electronic|99
4,Slot 5,21: Celestial Voyage|Electronic|95,15: Golden Ember|Rock|61,16: Mystic Rhythms|Classical|78,11: Phantom Groove|Hip-Hop|47,14: Synthwave Saints|Rock|94
5,Slot 6,4: The Silver Owls|Classical|85,28: The Polyrhythm Syndicate|Jazz|66,20: The Sonic Drifters|Rock|88,3: Neon Reverie|Electronic|100,26: Cloud Nine Collective|Pop|97
6,Slot 7,13: Lunar Spectrum|Rock|99,25: Rhythm Alchemy|Jazz|94,27: Hypnotic Echoes|Rock|77,5: Echo Chamber|Electronic|98,33: Cosmic Frequency|Rock|53


### 3. Defining fitness

" The  quality  of  a  festival  lineup  is  determined  by  balancing  __`three  equally`__  important 
objectives** ,  each  contributing  to  an  overall  score.  Since  these  objectives  have  different 
scales,  ** they  must  be  normalized  to  the  same  range  (between  0  and  1)  to  ensure  they 
contribute equally to the final fitness score.
<br>
 These are the objectives: 
 
__`Prime  Slot  Popularity`__  :  The  most  popular  artists  should  be  scheduled  in  the  prime 
slots  (**the  last  time  slot  on  each  stage**).  This  score  is  calculated  by  normalizing  the 
total  popularity  of  artists  in  prime  slots  against  the  maximum  possible  total  popularity 
(e.g. if only most popular artists - score 100 - were scheduled on the prime slot). 

__`Genre  Diversity`__  :  A  diverse  distribution  of  **genres  among  stages  in  each  time  slot**
improves  the  festival  experience.  This  score  is  determined  by  normalizing  the 
number  of  unique  genres  in  each  slot  relative  to  the  maximum  possible  number  of 
unique  genres  (e.g.  if  only  different  genres  were  scheduled  in  that  time  slot)  .  Then 
you take the average among time slots. 

__`Conflict  Penalty`__  : ** Fan  conflicts  occur  when  artists  with  overlapping  audiences 
perform  at  the  same  time  on  different  stages**. 
 This  score  is  calculated  by  normalizing 
the  total  conflict  value  in  each  slot  against  the  worst  possible  conflict  scenario  (e.g. 
where  all  artists  with  maximum  conflict  are  scheduled  together).  Then  you  take  the 
average  among  time  slots.  Since  conflicts  negatively  impact  the  lineup,  this  is  a 
penalization  score. 
"

__`Prime  Slot  Popularity`__  : 

In [11]:
data._get_max_popularity()

493

In [9]:
popularity_sum=data._get_sum_popularity([3,23,5,26])
popularity_sum

394

__`Genre  Diversity`__  : 

- Get maximum number of genres per slot, can not be greater than the total number of stages, which is 5

In [12]:
#Genre  Diversity
data._get_max_distinct_genre_per_slot()

5

- Get the number of distinct genres having a list of artistsIds


In [13]:
data._get_count_distinct_genres(artists_ids_list=[0,1,10])

3

__`Penalty Conflict`__  : 

In [72]:
start=time.time()
sum_conflicts=data._get_sum_conflits(list(range(34)))
print(f" {sum_conflicts} Dif in time {start-time.time()}")

 274.1 Dif in time -0.0007872581481933594


In [73]:
start=time.time()
sum_conflicts=data._get_sum_conflits_loop(list(range(34)))
print(f" {sum_conflicts} Dif in time {start-time.time()}")

 274.1000000000001 Dif in time -0.0020122528076171875


__`Penalty Conflict`__  - Max conflict: 

In [175]:
data._get_max_worst_conflict_loop()


np.float64(5.0)

In [156]:
data._get_max_worst_conflict()

np.float64(5.0)

In [164]:
lineup.repr

'0709162406192703102528230800183217203304313022293411050115022621131412'

In [174]:
lineup._get_slot_repr_list(1)

[19, 27, 3, 10, 25]

In [165]:
lineup._get_repr_list()

[7,
 9,
 16,
 24,
 6,
 19,
 27,
 3,
 10,
 25,
 28,
 23,
 8,
 0,
 18,
 32,
 17,
 20,
 33,
 4,
 31,
 30,
 22,
 29,
 34,
 11,
 5,
 1,
 15,
 2,
 26,
 21,
 13,
 14,
 12]

In [166]:
lineup.fitness()

[7, 9, 16, 24, 6]
[19, 27, 3, 10, 25]
[28, 23, 8, 0, 18]
[32, 17, 20, 33, 4]
[31, 30, 22, 29, 34]
[11, 5, 1, 15, 2]
[26, 21, 13, 14, 12]
